## 图的保存与加载

In [1]:
import tensorflow as tf

D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\tensorf

In [2]:
# 图的保存
v = tf.Variable(1.0, name='my_variable')
with tf.Session() as sess:
    tf.train.write_graph(sess.graph_def, './tfmodel', 'test_pb.pb', as_text=False)

In [3]:
# 图的加载
with tf.Session() as sess:
    with tf.gfile.GFile('./tfmodel/test_pb.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='tf.graph')
        print(graph_def)

node {
  name: "my_variable/initial_value"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "my_variable"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "my_variable/Assign"
  op: "Assign"
  input: "my_variable"
  input: "my_variable/initial_value"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@my_variable"
      }
    }
  }
  attr {
    key: "use_locking"
    value {
      b: true
    }
  }
  attr {
    key: "validate_shape"
    value {
      b: tr

## 以Inception模型为例，通过图文件导入模型

### 11.1导入Inception模型

### 导入库

In [4]:
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
import scipy.misc
import tensorflow as tf
import matplotlib

### 创建图和会话

In [6]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph = graph)


### 导入模型
Tensorflow提供了一种以'.pb'为扩展名的文件，可以事先将模型导入到pb文件中，再在需要的时候导出

In [7]:
model_fn = 'tensorflow_inception_graph.pb' # 导入Inception网络
# tensorflow_inception_graph.pb文件的下载：
# https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip

with tf.gfile.GFile(model_fn,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [8]:
# 定义输入图像的占位符
t_input = tf.placeholder(np.float32, name='input')

# 图像预处理——减均值
imagenet_mean = 117.0 # 在训练Inception模型时做了减均值预处理，此处也需减同样的均值以保持一致

# 图像预处理——增加维度
# 图像数据格式一般是（height, width, channels）,为同时将多张图片输入网络而在前面增加一维变为（batch,height,width,channel）
t_preprocessed = tf.expand_dims(t_input - imagenet_mean, 0)

# 导入模型并将经预处理的图像送入网络
tf.import_graph_def(graph_def, {'input':t_preprocessed})

### 找出卷积层或可视化的卷积层

In [9]:
layers = [op.name for op in graph.get_operations() if op.type == 'Conv2D']
# 输出卷积层层数
print('Number of layers', len(layers))

Number of layers 59


In [10]:
# 输出所有卷积层名称
print(layers)

['import/conv2d0_pre_relu/conv', 'import/conv2d1_pre_relu/conv', 'import/conv2d2_pre_relu/conv', 'import/mixed3a_1x1_pre_relu/conv', 'import/mixed3a_3x3_bottleneck_pre_relu/conv', 'import/mixed3a_3x3_pre_relu/conv', 'import/mixed3a_5x5_bottleneck_pre_relu/conv', 'import/mixed3a_5x5_pre_relu/conv', 'import/mixed3a_pool_reduce_pre_relu/conv', 'import/mixed3b_1x1_pre_relu/conv', 'import/mixed3b_3x3_bottleneck_pre_relu/conv', 'import/mixed3b_3x3_pre_relu/conv', 'import/mixed3b_5x5_bottleneck_pre_relu/conv', 'import/mixed3b_5x5_pre_relu/conv', 'import/mixed3b_pool_reduce_pre_relu/conv', 'import/mixed4a_1x1_pre_relu/conv', 'import/mixed4a_3x3_bottleneck_pre_relu/conv', 'import/mixed4a_3x3_pre_relu/conv', 'import/mixed4a_5x5_bottleneck_pre_relu/conv', 'import/mixed4a_5x5_pre_relu/conv', 'import/mixed4a_pool_reduce_pre_relu/conv', 'import/mixed4b_1x1_pre_relu/conv', 'import/mixed4b_3x3_bottleneck_pre_relu/conv', 'import/mixed4b_3x3_pre_relu/conv', 'import/mixed4b_5x5_bottleneck_pre_relu/conv',

In [11]:
# 输出指定卷积层的参数
name1 = 'mixed4d_3x3_bottleneck_pre_relu'
print('shape of %s:%s' % (name1, str(graph.get_tensor_by_name('import/'+name1+':0').get_shape())))

name2 = 'mixed4e_5x5_bottleneck_pre_relu'
print('shape of %s:%s' % (name2, str(graph.get_tensor_by_name('import/'+name2+':0').get_shape())))

shape of mixed4d_3x3_bottleneck_pre_relu:(?, ?, ?, 144)
shape of mixed4e_5x5_bottleneck_pre_relu:(?, ?, ?, 32)


卷积层格式一般是（batch, height, width, channel），因为此时还不清楚输入图像的数量及大小，所以前三个维度是不确定的，显示“？”。由于导入的是已训练好的模型，所以指定的卷积层的通道数量是固定的

## 11.2生成原始的Deep Dream图像

### 定义相关函数

In [12]:
from PIL import Image

# 把一个numpy.ndarray保存成图像文件
def savearray(img_array, img_name):
    scipy.misc.toimage(img_array).save(img_name)
    # 报错：module 'scipy.misc' has no attribute 'toimage',scipy在1.2.0版本后折旧，需要安装1.1.0版本
    print('img saved:%s' % img_name)
    
    
# 渲染函数
def render_naive(t_obj, img0, iter_n=20, step=1.0):
    # t_obj:是layer_output[:,:,:,channel],即卷积层某个通道的值
    # img0:初始图像（噪声图像）
    # iter_n：迭代次数
    # step：用于控制每次迭代步长，可以看作学习率
    t_score = tf.reduce_mean(t_obj)
    # t_score是t_obj的平均值
    # 由于我们的目标是调整输入图像使卷积层激活值尽可能大
    # 即最大化t_score
    # 为达到此目标，可使用梯度下降
    # 计算t_score对t_input的梯度
    t_grad = tf.gradients(t_score, t_input)[0]
    
    img = img0.copy() # 赋值新图像可避免影响原图像的值
    for i in range(iter_n):
        # 在sess中计算梯度，以及当前的t_score
        g, score = sess.run([t_grad, t_score], {t_input:img})
        # 对img应用梯度
        # 首先对梯度进行归一化处理
        g /= g.std() + 1e-8
        # 将正规化处理后的梯度应用在图像上，step用于控制每次迭代步长，此处为1.0
        img += g * step
        print('iter:%d' %(i+1), 'score(mean)=%f'%score)
    # 保存图片
    savearray(img, 'naive_deepdream.jpg')
    # matplotlib.image.imsave('naive_deepdream.jpg', img)
    #print(img.shape)
    #im = Image.fromarray(np.trunc(img))
    #im.save('naive_deepdream.jpg')

### 11.2.1 通过单通道特征生成DeepDream图像


In [13]:
# 定义卷积层、通道数，并取出对应的tensor
name = 'mixed4d_3x3_bottleneck_pre_relu' #(?,?,?,144)
channel = 139
# 'mixed4d_3x3_bottleneck_pre_relu'共144个通道
# 这里选任意通道（0~143之间任意整数）进行最大化

layer_output = graph.get_tensor_by_name('import/%s:0' % name)
# layer_output[:,:,:,channel]即可表示该卷积层的第140个通道

# 定义图像噪声
img_noise = np.random.uniform(size=(224,224,3)) + 100.0

# 调用render_naive函数渲染
render_naive(layer_output[:,:,:,channel], img_noise, iter_n=20)

#保存并显示图片
im = PIL.Image.open('naive_deepdream.jpg')
im.show()
im.save('naive_single_chn.jpg')

iter:1 score(mean)=-19.951572
iter:2 score(mean)=-27.021935
iter:3 score(mean)=9.321432
iter:4 score(mean)=89.999641
iter:5 score(mean)=157.510544
iter:6 score(mean)=229.526306
iter:7 score(mean)=293.511505
iter:8 score(mean)=347.090118
iter:9 score(mean)=401.760529
iter:10 score(mean)=455.130829
iter:11 score(mean)=493.631134
iter:12 score(mean)=529.031006
iter:13 score(mean)=565.489807
iter:14 score(mean)=612.621033
iter:15 score(mean)=636.746582
iter:16 score(mean)=670.305664
iter:17 score(mean)=697.119202
iter:18 score(mean)=725.442749
iter:19 score(mean)=751.152344
iter:20 score(mean)=765.506775
img saved:naive_deepdream.jpg


D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """


### 11.2.2 利用较低层单通道卷积特征生成deepdream图像

In [15]:
# 定义卷积层、通道数，并取出对应的tensor
name3 = 'mixed3a_3x3_bottleneck_pre_relu'
layer_output = graph.get_tensor_by_name('import/%s:0' % name3)
print('shape of %s:%s' % (name3, str(graph.get_tensor_by_name('import/'+name3+':0').get_shape())))

shape of mixed3a_3x3_bottleneck_pre_relu:(?, ?, ?, 96)


In [16]:
# 定义噪声图像
img_noise = np.random.uniform(size=(224,224,3)) + 100.0

# 调用render_naive函数渲染
channel = 86
render_naive(layer_output[:,:,:,channel], img_noise, iter_n=20)

# 保存并显示图片
im = PIL.Image.open('naive_deepdream.jpg')
im.show()
im.save('shallow_single_chn.jpg')

iter:1 score(mean)=5.881767
iter:2 score(mean)=47.837353
iter:3 score(mean)=134.954086
iter:4 score(mean)=205.076019
iter:5 score(mean)=251.079819
iter:6 score(mean)=280.931946
iter:7 score(mean)=303.147644
iter:8 score(mean)=319.365906
iter:9 score(mean)=332.227203
iter:10 score(mean)=342.304413
iter:11 score(mean)=351.176208
iter:12 score(mean)=358.733734
iter:13 score(mean)=364.863586
iter:14 score(mean)=370.420715
iter:15 score(mean)=375.565887
iter:16 score(mean)=379.834534
iter:17 score(mean)=383.611938
iter:18 score(mean)=387.163513
iter:19 score(mean)=390.271515
iter:20 score(mean)=393.273224
img saved:naive_deepdream.jpg


D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """


### 11.2.3 利用较高层单通道卷积特征生成deepdream图像


In [17]:
# 定义卷积层、通道数，并取出对应的tensor
name4 = 'mixed5b_5x5_pre_relu'
layer_output = graph.get_tensor_by_name('import/%s:0' % name4)
print('shape of %s:%s' % (name4, str(graph.get_tensor_by_name('import/'+name4+':0').get_shape())))

shape of mixed5b_5x5_pre_relu:(?, ?, ?, 128)


In [18]:
# 定义噪声图像
img_noise = np.random.uniform(size=(224,224,3)) + 100.0

# 调用render_naive函数渲染
channel = 118
render_naive(layer_output[:,:,:,channel], img_noise, iter_n=20)

# 保存并显示图片
im = PIL.Image.open('naive_deepdream.jpg')
im.show()
im.save('deep_single_chn.jpg')

iter:1 score(mean)=-7.691501
iter:2 score(mean)=-7.414789
iter:3 score(mean)=-3.892986
iter:4 score(mean)=3.874886
iter:5 score(mean)=9.556870
iter:6 score(mean)=20.077684
iter:7 score(mean)=33.010136
iter:8 score(mean)=42.359364
iter:9 score(mean)=48.868275
iter:10 score(mean)=54.380367
iter:11 score(mean)=61.392803
iter:12 score(mean)=71.589645
iter:13 score(mean)=83.234322
iter:14 score(mean)=90.993347
iter:15 score(mean)=101.201355
iter:16 score(mean)=112.333923
iter:17 score(mean)=116.692314
iter:18 score(mean)=123.983963
iter:19 score(mean)=132.368805
iter:20 score(mean)=137.970184
img saved:naive_deepdream.jpg


D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """


### 11.2.4利用多通道卷积生成deepdream图像

### 11.2.5 利用所有通道特征生成deepdream图像

In [19]:
# 定义卷积层，并取出对应的tensor
name = 'mixed4d_3x3_bottleneck_pre_relu' #(?,?,?,144)
layer_output = graph.get_tensor_by_name('import/%s:0' % name)

# 定义图像噪声
img_noise = np.random.uniform(size=(224,224,3)) + 100.0

# 调用render_naive函数渲染
render_naive(layer_output, img_noise, iter_n=20)# 不指定特定通道，即表示利用所有通道特征

#保存并显示图片
im = PIL.Image.open('naive_deepdream.jpg')
im.show()
im.save('all_chn.jpg')

iter:1 score(mean)=-6.643812
iter:2 score(mean)=-10.414528
iter:3 score(mean)=-5.787938
iter:4 score(mean)=0.727738
iter:5 score(mean)=6.897670
iter:6 score(mean)=12.349975
iter:7 score(mean)=15.036036
iter:8 score(mean)=19.979305
iter:9 score(mean)=24.169178
iter:10 score(mean)=27.378139
iter:11 score(mean)=28.527636
iter:12 score(mean)=32.889164
iter:13 score(mean)=36.075706
iter:14 score(mean)=37.162251
iter:15 score(mean)=41.961651
iter:16 score(mean)=46.405144
iter:17 score(mean)=46.753391
iter:18 score(mean)=50.417381
iter:19 score(mean)=51.870598
iter:20 score(mean)=55.001865
img saved:naive_deepdream.jpg


D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """


### 11.2.6 利用背景图像生成deepdream图像

In [20]:
# 定义卷积层，并取出对应的tensor
name = 'mixed4c'
layer_output = graph.get_tensor_by_name('import/%s:0' % name)
print(layer_output)

Tensor("import/mixed4c:0", shape=(?, ?, ?, 512), dtype=float32, device=/device:CPU:0)


In [26]:
# 用一张背景图像（而不是随机噪声图像）作为起点对图像进行优化
img_test = PIL.Image.open('mountain.jpg')

# 调用render_naive函数渲染
render_naive(layer_output, img_test, iter_n=20)# 不指定特定通道，即表示利用所有通道特征

#保存并显示图片
im = PIL.Image.open('naive_deepdream.jpg')
im.show()
im.save('mountain_naive.jpg')


iter:1 score(mean)=13.039110
iter:2 score(mean)=19.383553
iter:3 score(mean)=26.859833
iter:4 score(mean)=33.929298
iter:5 score(mean)=39.912254
iter:6 score(mean)=45.326698
iter:7 score(mean)=50.238174
iter:8 score(mean)=54.467319
iter:9 score(mean)=58.327110
iter:10 score(mean)=61.892525
iter:11 score(mean)=65.174721
iter:12 score(mean)=68.301033
iter:13 score(mean)=70.989845
iter:14 score(mean)=73.654099
iter:15 score(mean)=76.258476
iter:16 score(mean)=78.235771
iter:17 score(mean)=80.552315
iter:18 score(mean)=82.473038
iter:19 score(mean)=84.483421
iter:20 score(mean)=86.126228
img saved:naive_deepdream.jpg


D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until


## 生成更高质量的Deep Dream图像

### 导入库与Inception模型

In [22]:
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
import scipy.misc
import tensorflow as tf

graph = tf.Graph()
model_fn = 'tensorflow_inception_graph.pb'
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.GFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='input')
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input - imagenet_mean, 0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})


D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### 定义相关函数

In [23]:
# 保存图像
def savearray(img_array, img_name):
    scipy.misc.toimage(img_array).save(img_name)
    print('img saved:%s'% img_name)
    
# 将图像放大ratio倍
def resize_ratio(img, ratio):
    min = img.min()
    max = img.max()
    img = (img-min) / (max - min) * 255
    img = np.float32(scipy.misc.imresize(img, ratio))
    img = img / 255*(max-min) + min
    return img

# 调整图像尺寸
def resize(img, hw):
    min = img.min()
    max = img.max()
    img = (img-min) / (max - min) * 255
    img = np.float32(scipy.misc.imresize(img, hw))
    img = img / 255*(max-min) + min
    return img

# 原始图像尺寸可能很大，从而导致内存耗尽问题
# 每次只对tile_size * tile_size大小的图像计算梯度，避免内存问题
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h, w = img.shape[:2]
    sx, sy = np.random.randint(sz, size=2)
    img_shift = np.roll(np.roll(img, sx,1), sy, 0)# 先在行上做整体移动，再在列上做整体移动
    grad = np.zeros_like(img)
    for y in range(0, max(h - sz//2, sz), sz):
        for x in range(0, max(w - sz//2, sz), sz):
            sub = img_shift[y:y+sz, x:x+sz]
            g = sess.run(t_grad, {t_input:sub})
            grad[y:y+sz, x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy, 0)
    

# 渲染函数,对图像进行拉普拉斯分解
def render_deepdream(t_obj, img0, 
                     iter_n=20, step=1.5, octave_n=4, octave_scale=1.4):
    # t_obj:是layer_output[:,:,:,channel],即卷积层某个通道的值
    # img0:初始图像（噪声图像）
    # iter_n：迭代次数
    # step：用于控制每次迭代步长，可以看作学习率
    # octave_n：金字塔层数
    # octave_scale：层与层之间的倍数
    t_score = tf.reduce_mean(t_obj)
    # t_score是t_obj的平均值
    # 由于我们的目标是调整输入图像使卷积层激活值尽可能大
    # 即最大化t_score
    # 为达到此目标，可使用梯度下降
    # 计算t_score对t_input的梯度
    t_grad = tf.gradients(t_score, t_input)[0]
    
    img = img0.copy() # 赋值新图像可避免影响原图像的值
    
    # 将图像进行金字塔分解，从而分为高频、低频部分
    octaves = []
    for i in range(octave_n - 1):
        hw = img.shape[:2]# 图像尺寸
        lo = resize(img, np.int32(np.float32(hw)/octave_scale))
        hi = img - resize(lo, hw)
        img = lo
        octaves.append(hi)
        
    # 首先生成低频的图像，再依次放大并加上高频
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]
            img = resize(img, hi.shape[:2]) + hi
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            img += g*(step/np.abs(g).mean()+1e-7)
            
    img = img.clip(0,255)
    savearray(img, 'mountain_deepdream.jpg')
    im = PIL.Image.open('mountain_deepdream.jpg').show()

生成以背景图像作为起点的DeepDream图像

In [27]:
name = 'mixed4c'
layer_output = graph.get_tensor_by_name('import/%s:0' % name)

img0 = PIL.Image.open('dog.jpg')
img0 = np.float32(img0)
render_deepdream(tf.square(layer_output), img0)

D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
D:\SoftWare\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until


img saved:mountain_deepdream.jpg
